# Наивный Байесовский Классификатор для классификации спам-сообщений

## Наивный байесовский классификатор by hand

### Загрузка библиотек и данных

In [1]:
# Импортируем необходимые библиотеки
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных

from sklearn.feature_extraction.text import CountVectorizer # векторизатор
from sklearn.metrics import accuracy_score, classification_report # метрики
from sklearn.model_selection import train_test_split # сплитование выборки
from sklearn.naive_bayes import MultinomialNB, ComplementNB # модели НБК

Прочитаем файл (разделителем здесь выступает символ табуляции).

In [2]:
sms_data = pd.read_csv('data/SMSSpamCollection.zip', header=None, sep='\t', names=['Label', 'SMS'])
sms_data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Посмотрим, сколько объектов каждого класса присутствует в датасете.

In [3]:
sms_data.groupby('Label').count()

,SMS
Label,
ham,4825
spam,747


### Предобработка данных

Удаляем символы, не являющиеся буквами, приводим тексты SMS к нижнему регистру, разбиваем строки на слова.

In [4]:
sms_data_clean = sms_data.copy()

In [5]:
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.replace('\W+', ' ', regex=True)

sms_data_clean['SMS'] = sms_data_clean['SMS'].str.replace('\s+', ' ', regex=True).str.strip()
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.lower()
sms_data_clean['SMS'] = sms_data_clean['SMS'].str.split()

sms_data_clean['SMS'].head()

0    [go, until, jurong, point, crazy, available, o...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, in, 2, a, wkly, comp, to, win, f...
3    [u, dun, say, so, early, hor, u, c, already, t...
4    [nah, i, don, t, think, he, goes, to, usf, he,...
Name: SMS, dtype: object

In [6]:
# Выводим доли классов в данных
sms_data_clean['Label'].value_counts() / sms_data_clean.shape[0] * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

### Разделение на обучающую и тестовую выборки

In [7]:
# Сэмплируем
train_data = sms_data_clean.sample(frac=0.8, random_state=42)
test_data = sms_data_clean.drop(train_data.index)
# Сбрасываем индексацию
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [8]:
# Считаем доли классов в треннировочной выборке
train_data['Label'].value_counts() / train_data.shape[0] * 100

ham     86.698071
spam    13.301929
Name: Label, dtype: float64

In [9]:
# Размер треннировочной выборки
train_data.shape

(4458, 2)

In [10]:
# Считаем доли классов в тестовой выборке
test_data['Label'].value_counts() / test_data.shape[0] * 100

ham     86.175943
spam    13.824057
Name: Label, dtype: float64

In [11]:
# Размер тестовой выборки
test_data.shape

(1114, 2)

Мы видим, что и в обучающей, и в тестовой выборке содержится примерно 86-87% спама – как и в нашем оригинальном датасете. Сэмплирование прошло вполне удачно.

### Список слов

Создаём список всех слов, встречающихся в обучающей выборке.

In [12]:
# Объединяем списки слов в сообщениях и проводим
# нашу коллекцию через преобразование во множество
vocabulary = list(set(train_data['SMS'].sum()))

In [13]:
# Образец результата
vocabulary[11:20]

['irritates',
 'laying',
 'your',
 'pink',
 'destiny',
 'brings',
 'tagged',
 'ambrith',
 'goodnite']

In [14]:
# Число итоговых уникальных строк
len(vocabulary)

7816

### Рассчитаем частоты слов

Для каждого SMS-сообщения посчитаем, сколько раз в нём встречается каждое слово.

In [15]:
%%time

word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()
], columns=vocabulary)

word_counts_per_sms.head()

Wall time: 33.7 s


,decking,parts,onwards,holiday,fulfil,cat,wasnt,atten,values,predictive,...,shakespeare,latr,city,downstem,impressed,79,quick,postcode,ors,talents
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Добавим частоты каждого слова в обучающий датасет.

In [16]:
train_data = pd.concat([train_data, word_counts_per_sms], axis=1)

In [17]:
train_data.head(3)

,Label,SMS,decking,parts,onwards,holiday,fulfil,cat,wasnt,atten,...,shakespeare,latr,city,downstem,impressed,79,quick,postcode,ors,talents
0,ham,"[squeeeeeze, this, is, christmas, hug, if, u, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[and, also, i, ve, sorta, blown, him, off, a, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[mmm, thats, better, now, i, got, a, roast, do...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Экскурс в теорию

Наивный байесовский классификатор (НБК, англ. Naive Bayes Classifier, NBC) решает задачу классификации объектов по типам. Большим преимуществом этого алгоритма является его простота, как идейная, так и алгоритмическая.

Наивная байесовская классификация — это достаточно простой вероятностный алгоритм, основанный на том, что все признаки модели независимы.

Если, например, мы говорим про задачу классификации спам-писем и обычных писем, то в этом контексте мы считаем, что каждое слово в сообщении не зависит от всех других слов, то есть каждое слово мы учитываем, не обращая внимания на контекст.

Алгоритм классификации выдаёт вероятность того, является письмо спамом или нет, основываясь на наборе слов в письме. Расчёт этой вероятности основан на формуле Байеса, а компоненты формулы рассчитываются на основе частот слов во всём наборе сообщений.

Прежде всего, возьмём формулу Байеса и применим её к нашей задаче:

$$P(Спам|w_1, w_2, ..., w_n) \propto P(Спам) \cdot \displaystyle\prod_{i=1}^{n} P(w_i | Спам)$$

Вероятность того, что письмо является спамом при условии, что в нём есть определённые слова (которые мы обозначили), пропорциональна произведению двух значений:
 - вероятности получения спама в целом (по сути, это доля спама в выборке);
 - произведения вероятностей, что в письме есть некоторое слово $w_i$, если письмо является спамом, для всех слов выборки.

Разберёмся с этим подробнее. Для каждого слова в сообщении мы рассчитываем вероятность того, что это слово окажется в спаме. В рамках нашей задачи рассматриваем следующие значения:
 - $P(spam)$ — вероятность, что случайно взятое письмо будет спамом (также это доля спам-сообщений в нашем наборе данных);
 - $P(w_i | spam)$ — вероятность того, что в сообщении будет определённое слово, если это письмо является спамом.

По той же логике можем определить:
 - $P(not spam)$ — доля сообщений, которые не являются спамом;
 - $P(w_i | not spam)$ — вероятность того, что в сообщении будет определённое слово, если это письмо не является спамом.

Теперь необходимо понять, как рассчитать вероятности каждого слова. Для этого в алгоритме используется следующая формула:

$$P(w_i | spam)=\frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$$

В этой формуле:
 - $N_{Vocabulary}$ — количество уникальных слов во всём наборе данных;
 - $N_{Spam}$ — общее количество слов в спам-сообщениях;
 - $N_{w_i|Spam}$ — количество повторов слова во всех спам-сообщениях;
 - $\alpha$ — коэффициент для случаев, когда слово в сообщении отсутствует в нашем наборе данных.

Кратко это можно объяснить так: вероятность того, что это слово встретится в спам сообщении, — это частота этого слова в «спамовой части» нашего набора данных (но с добавлением «сглаживания», чтобы учитывать ситуации, когда попадаются слова, которых не было в обучающей выборке).

Также эта же формула (но с другими значениями) верна для вероятности того, что слово принадлежит не спам-сообщениям.

### Значения для формулы Байеса

Посчитаем необходимые значения для формулы Байеса.

In [18]:
alpha = 1

Nvoc = len(vocabulary)
Pspam = train_data['Label'].value_counts()['spam'] / train_data.shape[0]
Pham = train_data['Label'].value_counts()['ham'] / train_data.shape[0]
Nspam = train_data.loc[train_data['Label'] == 'spam', 'SMS'].apply(len).sum()
Nham = train_data.loc[train_data['Label'] == 'ham', 'SMS'].apply(len).sum()

In [19]:
def p_w_spam(word):
    """
    Функция для расчета вероятности присутствия слова в спаме.
    """
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'spam', word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1

def p_w_ham(word):
    """
    Функция для расчета вероятности присутствия слова в письме.
    """
    if word in train_data.columns:
        return (train_data.loc[train_data['Label'] == 'ham', word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1

### Готовим алгоритм классификации

In [20]:
def classify(message):
    """
    Функция-классификатор письма на основе рассчитанных вероятностей.
    """
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'ham'

### Используем тестовые данные

In [21]:
%%time
test_data['predicted'] = test_data['SMS'].map(classify)

Wall time: 9.92 s


In [22]:
test_data.head()

,Label,SMS,predicted
0,ham,"[u, dun, say, so, early, hor, u, c, already, t...",ham
1,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",ham
2,spam,"[freemsg, hey, there, darling, it, s, been, 3,...",ham
3,spam,"[had, your, mobile, 11, months, or, more, u, r...",spam
4,ham,"[oh, k, i, m, watching, here]",ham


In [23]:
correct = (test_data['predicted'] == test_data['Label']).sum() / test_data.shape[0]
print(f"Правильных предсказаний {correct * 100:3f} %")

Правильных предсказаний 98.384201 %


In [24]:
# Выведем список сообщений (их немного) с неверной классификацией
test_data.loc[test_data['predicted'] != test_data['Label']]

,Label,SMS,predicted
2,spam,"[freemsg, hey, there, darling, it, s, been, 3,...",ham
96,ham,"[waiting, for, your, call]",spam
182,ham,"[26th, of, july]",spam
269,spam,"[sms, ac, jsco, energy, is, high, but, u, may,...",ham
479,spam,"[this, message, is, brought, to, you, by, gmw,...",ham
528,spam,"[how, come, it, takes, so, little, time, for, ...",ham
576,spam,"[do, you, ever, notice, that, when, you, re, d...",ham
652,spam,"[rct, thnq, adrian, for, u, text, rgds, vatian]",ham
679,spam,"[life, has, never, been, this, much, fun, and,...",ham
687,ham,"[wiskey, brandy, rum, gin, beer, vodka, scotch...",spam


Посчитаем все метрики.

In [25]:
y_test_classify_pred = test_data['predicted']
y_test = test_data['Label']

# Вывод отчета о метриках тестовой выборки
print(classification_report(y_test, y_test_classify_pred, digits=3))

              precision    recall  f1-score   support

         ham      0.987     0.995     0.991       960
        spam      0.966     0.916     0.940       154

    accuracy                          0.984      1114
   macro avg      0.976     0.955     0.965      1114
weighted avg      0.984     0.984     0.984      1114



## Наивный байесовский классификатор в sklearn

Ура, мы реализовали наивный байесовский классификатор с нуля!
А теперь посмотрим, как то же самое можно сделать с помощью библиотеки scikit-learn.

### Загрузка и предобработка данных

Прочитаем заново csv-файл и предобработаем данные. Разбивать сообщения на слова в этот раз не нужно, мы сделаем это далее с помощью встроенных инструментов

In [26]:
df = pd.read_csv(
    "data/SMSSpamCollection.zip", header=None, sep="\t", names=["Label", "SMS"]
)

# Также предобработаем содержимое сообщений
df["SMS"] = df["SMS"].str.replace(r"\W+", " ", regex=True).str.lower()
df['SMS'] = df['SMS'].str.replace('\s+', ' ', regex=True).str.strip()
df['SMS'] = df['SMS'].str.lower()
df.head()

,Label,SMS
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives arou...


### Векторизация и сплитование

Преобразуем строки в векторный вид – то есть, снова создадим таблицу с частотами слов. Но в этот раз воспользуемся встроенным в sklearn классом CountVectorizer().

In [27]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["SMS"])
y = df["Label"]

print(X.shape, y.shape)

(5572, 8713) (5572,)


Любопытно: в этот раз число столбцов больше почти на тысячу. CountVectorizer не настолько прост, как наш ручной алгоритм.

С помощью функции `train_test_split` из scikit-learn разобьём выборку на обучающую и тестовую в пропорции 80/20. И здесь мы уже можем прописать стратификацию явно.

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

### Обучение моделей

В библиотеке sklearn есть два подходящих нам байесовских классификатора:
 - MultinomialNB  — работает с категориальными признаками, текстами и несбалансированными выборками;
 - ComplementNB — улучшенная версия MultinomialNB, стабильно показывает более высокое качество в задачах классификации текстов.

In [29]:
# Инициализируем объект MultinomialNB
mnb = MultinomialNB()
# Обучим
mnb.fit(X_train, y_train)
# Сделаем предсказания классов объектов в обоих выборках
y_train_mnb_pred = mnb.predict(X_train)
y_test_mnb_pred = mnb.predict(X_test)

# Вывод отчета о метриках треннировочной выборки
print(classification_report(y_train, y_train_mnb_pred, digits=3))

# Вывод отчета о метриках тестовой выборки
print(classification_report(y_test, y_test_mnb_pred, digits=3))

              precision    recall  f1-score   support

         ham      0.995     0.997     0.996      3859
        spam      0.980     0.968     0.974       598

    accuracy                          0.993      4457
   macro avg      0.987     0.983     0.985      4457
weighted avg      0.993     0.993     0.993      4457

              precision    recall  f1-score   support

         ham      0.992     0.984     0.988       966
        spam      0.904     0.946     0.925       149

    accuracy                          0.979      1115
   macro avg      0.948     0.965     0.956      1115
weighted avg      0.980     0.979     0.980      1115



In [30]:
# Инициализируем объект MultinomialNB
cnb = ComplementNB()
# Обучим
cnb.fit(X_train, y_train)
# Сделаем предсказания классов объектов в обоих выборках
y_train_cnb_pred = cnb.predict(X_train)
y_test_cnb_pred = cnb.predict(X_test)

# Вывод отчета о метриках треннировочной выборки
print(classification_report(y_train, y_train_cnb_pred, digits=3))

# Вывод отчета о метриках тестовой выборки
print(classification_report(y_test, y_test_cnb_pred, digits=3))

              precision    recall  f1-score   support

         ham      0.996     0.989     0.992      3859
        spam      0.930     0.973     0.951       598

    accuracy                          0.987      4457
   macro avg      0.963     0.981     0.972      4457
weighted avg      0.987     0.987     0.987      4457

              precision    recall  f1-score   support

         ham      0.995     0.969     0.982       966
        spam      0.828     0.966     0.892       149

    accuracy                          0.969      1115
   macro avg      0.911     0.968     0.937      1115
weighted avg      0.972     0.969     0.970      1115



## Итог

Наш ручной алгоритм наивного байесовского классификатора справился с задачей весьма неплохо. Даже больше – он показал себя лучше, чем библиотечные модели с дефолтными параметрами. Однако последние мы еще можем оптимизировать, но это мы сделаем в следующий раз.